In [2]:
# !pip install -qU pip wheel
# !pip install -q -r ../requirements.txt
import sys
sys.path.append('..')
from ballitoreproject import *
# !pip install spacy en_core_web_sm
# !python -m spacy download en_core_web_sm

In [2]:
get_place_data

,sender,recipient,date,location from,location to,notes,box,datetime,dateyear,is_journal,txt,year,decade,supplemental_ids,num_letters,num_words
id,,,,,,,,,,,,,,,,
mss4-b1-f3-005,Abraham Shackleton I,Richard Shackleton,1744,,Dublin,extract,1,1744-07-13,1744.0,False,"P.7\t“Keep thy mind as quiet as possible, so t...",1744,1740,,1,37
mss4-b1-f3-005-006,Abraham Shackleton I,Richard Shackleton,1748,,Dublin,,1,1748-07-13,1748.0,False,"“We are very desirous of seeing thee at home, ...",1748,1740,,1,173
mss4-b1-f3-007,Richard Shackleton,Abraham Shackleton I,11/1/1755,Dublin,,,1,1755-11-01,1755.0,False,P. 25\t\t\t\t\t\t“Dublin 1st of Eleventhmonth ...,1755,1750,,1,73
mss4-b1-f3-008,Abraham Shackleton I,Richard Shackleton,6/2/1760,London,,,1,1760-06-02,1760.0,False,"P. 53\t\t\t\t\t\t“London, 2nd of Sixthmonth 17...",1760,1760,,1,43
mss4-b1-f3-008-010,Abraham Shackleton I,Richard Shackleton,9/18/1762,Cooladine,,,1,1762-09-18,1762.0,False,To the memorable visit paid to the meetings fo...,1762,1760,,1,180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
mss4-b12-f64-013,,,,,,remarks to monthly meeting,12,NaT,0.0,False,To Chas. Marqs. Cornwallis Ld Lieut. Gen & Ger...,0,0,,1,267
mss4-b12-f64-014,,,,,,Quakers' address to Lord Cornwallis,12,NaT,0.0,False,While he spake\nCurious they mark'd the Damsel...,0,0,,1,230
mss4-b12-f64-015,,,,,,poetry extracts,12,NaT,0.0,False,Arrivals in 1822\n1st month\nJacob Greene - Wi...,0,0,,1,28


In [3]:
import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")

def get_named_places(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Extract named entities labeled as GPE (Geopolitical Entity)
    places = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
    
    return places


In [4]:
get_named_places('This will be handed thee I suppose by Cd Morgan who intends for Clonmel tomorrow')

['Clonmel']

In [5]:
df_smpl['places']=[get_named_places(txt) for txt in tqdm(df_smpl.txt)]
df_smpl['places']

100%|██████████| 2504/2504 [04:34<00:00,  9.12it/s]


id
mss4-b1-f3-005                                               []
mss4-b1-f3-005-006                                         [us]
mss4-b1-f3-007                                           [R.S.]
mss4-b1-f3-008                                 [London, Israel]
mss4-b1-f3-008-010    [the Nation of Ireland, Ninthmonth, A.S.]
                                        ...                    
mss4-b12-f64-013                         [Ireld, Dublin, Matys]
mss4-b12-f64-014            [Damsel, s[unclear]ol[/unclear]emn]
mss4-b12-f64-015                                             []
mss4-b12-f64-016                                     [Spelling]
mss4-b12-f7-001                           [Margt, Dublin, Thou]
Name: places, Length: 2504, dtype: object

In [80]:
all_places = pd.Series(place for places in df_smpl.places for place in places).value_counts()
top_places=all_places.head(100)
top_places

Dublin     913
London     474
Ireland    378
Cousin     278
England    239
          ... 
Vienna      10
Scotch      10
York        10
Rushton      9
Island       9
Name: count, Length: 100, dtype: int64

In [72]:
# !pip install geopy
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="ballitoreproject")

In [73]:
loc=geolocator.geocode('Clonmel')
loc

Location(Clonmel, County Tipperary, Munster, Éire / Ireland, (52.355, -7.703889, 0.0))

In [74]:
loc.raw

{'place_id': 270243476,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 14613785,
 'lat': '52.355',
 'lon': '-7.703889',
 'class': 'place',
 'type': 'town',
 'place_rank': 18,
 'importance': 0.4531090656363466,
 'addresstype': 'town',
 'name': 'Clonmel',
 'display_name': 'Clonmel, County Tipperary, Munster, Éire / Ireland',
 'boundingbox': ['52.3440567', '52.3737368', '-7.7509356', '-7.6562234']}

In [75]:
geolocator.geocode('Clonmel', exactly_one=False)

[Location(Clonmel, County Tipperary, Munster, Éire / Ireland, (52.355, -7.703889, 0.0)),
 Location(Clonmel, Finglas East C ED, Dublin, County Dublin, Leinster, Éire / Ireland, (53.391658050000004, -6.273465224256768, 0.0)),
 Location(Clonmel, Illinois Township, Sedgwick County, Kansas, United States, (37.5627944, -97.5539369, 0.0)),
 Location(Clonmel, Clonbullogue ED, The Municipal District of Edenderry, County Offaly, Leinster, Éire / Ireland, (53.261587399999996, -7.064513297225947, 0.0)),
 Location(Clonmel, Parsonage, Saint Mary, Middlesex County, Jamaica, (18.269558, -76.8750394, 0.0)),
 Location(Clonmel, Saint Mary, Middlesex County, Jamaica, (18.247367, -76.849494, 0.0))]

In [76]:
geolocator.geocode('Waterford', exactly_one=False)

[Location(Waterford, County Waterford, Munster, Éire / Ireland, (52.2609997, -7.1119081, 0.0)),
 Location(County Waterford, Munster, Éire / Ireland, (52.1509131, -7.657798180652727, 0.0)),
 Location(Waterford, Southeastern Connecticut Planning Region, Connecticut, United States, (41.358659, -72.1519367, 0.0)),
 Location(Waterford, Stanislaus County, California, United States, (37.6413202, -120.760483, 0.0)),
 Location(Waterford, Caledonia County, Vermont, 05848, United States, (44.38813075, -71.95714833120239, 0.0)),
 Location(Village of Waterford, Town of Waterford, Saratoga County, New York, United States, (42.7925778, -73.6812293, 0.0)),
 Location(Waterford, Racine County, Wisconsin, United States, (42.7630722, -88.2142563, 0.0)),
 Location(Waterford, Oxford County, Maine, 04088, United States, (44.1816057, -70.7169424, 0.0)),
 Location(Town of Waterford, Saratoga County, New York, 12188, United States, (42.7925778, -73.6812293, 0.0)),
 Location(Waterford, Digby Municipal District, 

In [77]:
# !pip install diskcache

In [78]:
import diskcache as dc
cache_obj = dc.Cache(os.path.join(PATH_DATA,'geocache'))

@cache_obj.memoize()
def get_place_data(placename):
    loc = geolocator.geocode(placename)
    if not loc: return {}
    outd = {**loc.raw}
    outd['lat']=float(outd['lat'])
    outd['lon']=float(outd['lon'])
    return outd

get_place_data('Clonmel')

{'place_id': 270243476,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
 'osm_type': 'relation',
 'osm_id': 14613785,
 'lat': 52.355,
 'lon': -7.703889,
 'class': 'place',
 'type': 'town',
 'place_rank': 18,
 'importance': 0.4531090656363466,
 'addresstype': 'town',
 'name': 'Clonmel',
 'display_name': 'Clonmel, County Tipperary, Munster, Éire / Ireland',
 'boundingbox': ['52.3440567', '52.3737368', '-7.7509356', '-7.6562234']}

In [84]:
# places_ld=[]
# for place,count in tqdm(list(all_places.items())):
#     place_d={'place':place, 'count':count, **get_place_data(place)}
#     places_ld.append(place_d)
# places_df=pd.DataFrame(places_ld).dropna()
# places_df

In [55]:
# !pip install folium

In [56]:
import pandas as pd
import folium

def create_map_with_markers(df):
    """
    Creates a folium map with markers sized proportionally to the count values in the DataFrame.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing 'place', 'count', 'lat', 'lon' columns.
    
    Returns:
    folium.Map: Interactive map with markers.
    """
    # Create a folium map centered around the mean latitude and longitude
    map_center = [df['lat'].mean(), df['lon'].mean()]
    fmap = folium.Map(location=map_center, zoom_start=2)

    # Define a function to scale the marker sizes
    def scale_size(count, min_count, max_count, min_size=3, max_size=15):
        return ((count - min_count) / (max_count - min_count)) * (max_size - min_size) + min_size

    # Get min and max count values for scaling
    min_count = df['count'].min()
    max_count = df['count'].max()

    # Add markers to the map
    for _, row in df.iterrows():
        marker_size = scale_size(row['count'], min_count, max_count)
        folium.CircleMarker(
            location=(row['lat'], row['lon']),
            radius=marker_size,
            popup=f"{row['place']}: {row['count']}",
            color='blue',
            fill=True,
            fill_color='blue'
        ).add_to(fmap)

    return fmap

fmap = create_map_with_markers(places_df)
fmap
